## Repo demonstration for replicating the results

Please make sure that you have installed packages listed in `requirements.txt`

If you want to monitor training process, run `tensorboard --logdir runs`

We worked with 400 x 400 images in our experiments, using a single GPU with 8GB of memory.

`pipeline.py` is the main script for training which you can call by certain arguments:
- --stage: `geometry` when learning the scene geometry, `style` when transferring the style.
- --prefix and --suffix: trained models will be saved under `ckpts/{prefix}_nerf_coarse_{suffix}.pt` and `ckpts/{prefix}_nerf_fine_{suffix}.pt`
- --img_wh: image resolution, `400 400` in our experiments.
- --num_epochs_density: number of epochs when learning scene geometry, `1` in our experiments.
- --batch_size: batch size, only relevant in the `geometry` stage, `1024` in our experiments.
- --num_epochs_style: number of epochs when transferring the style, `1` in our experiments.
- --style_dir: path to the styling image, one of the following options: `starry_night.jpg`, `wheat_field.jpg`, `italian_futurism.jpg`
- --style_mode: the approach to take in style transfer stage, one of the following options: `small` (refers to simple method, takes around 5 mins), `patch` (refers to patched method, takes around 30 mins), `memory_saving` (refers to no patch method, takes around 50 mins)
- --coarse_path and --fine_path: when transferring the style, path to coarse and fine density models learned in Stage 1
- --lr: learning rate, in our experiments `5e-4` for `geometry` stage, `1e-3` for `style` stage with `small` or `memory_saving` mode, `1e-4` for `patch` mode

In [2]:
# Stage 1: Train the geometry MLP.
# Estimated runtime: 1 hour
# You can increase the batch size if you have access to more memory to speed up training.

!python pipeline.py --stage geometry --prefix test --suffix density --dataset_name blender --root_dir datasets/nerf_synthetic/lego --img_wh 400 400 --num_epochs_density 1 --batch_size 1024 --lr 5e-4

Starting epoch 1 to learn density...


In [3]:
# Stage 2: Train the style MLP with the starry night image and in memory saving mode. 
# Estimated runtime: 50 mins

!python pipeline.py --stage style --prefix test --suffix style --style_dir starry_night.jpg --style_mode memory_saving --coarse_path ckpts/test_nerf_coarse_density.pt --fine_path ckpts/test_nerf_fine_density.pt --dataset_name blender --root_dir datasets/nerf_synthetic/lego --img_wh 400 400 --num_epochs_style 1 --lr 1e-3

Mode: Memory saving (No patch), Starting epoch 1 to learn style...


In [4]:
# Create and save a gif out of the rendered images from 200 camera poses in test dataset under ./starry_night_memory_saving.gif
# Estimated runtime: 1 hour

!python eval.py --dataset_name blender --root_dir datasets/nerf_synthetic/lego --scene_name lego --img_wh 400 400 --coarse_path ckpts/test_nerf_coarse_style.pt --fine_path ckpts/test_nerf_fine_style.pt --gif_name starry_night_memory_saving

Mean PSNR : 13.63



100%|##########| 200/200 [04:07<00:00,  1.24s/it]


In [5]:
# Display the saved gif

from IPython.core.display import display, HTML
display(HTML('<h1><img src="starry_night_memory_saving.gif" width="400" align="center"></h1>'))